## CProfile

Executa e calcula o desempenho e consume de memória com o cprofile.
 
### Linha de comando:
***

Gerar o cprofile do seu código

* `python -m cProfile -o output.stats uvicorn ...`

* `python -m pstats output.stats`

    - stats N
    - sort <params> (filename, tottime, cumtime, ...)
    - quit

Com o output.stats gere uma imagem com a arvore de chamadas

* `pip install gprof2dot`
* `gprof2dot -f pstats output.stats | dot -T png -o output.png`

Verifique sua aplicação em tempo real

* `pip install py-spy`
* `py-spy top -- python uvicorn ...`

profile com corotines

* `pip install pyinstrument`
* `pyinstrument uvicorn ...`

***
### Parâmetros:
***

* **ncalls**: Números de chamadas do método

* **tottime**: Tempo total de execução do método sem as chamadas internas a outros métodos * ncalls
* **percall**: Tempo total de execução do método sem as chamadas internas a outros métodos
* **cumtime**: Tempo de espera que esse método teve com suas chamadas internas a outros métodos * ncalls
* **percall**: Tempo de espera que esse método teve com suas chamadas internas a outros métodos
* **filename**: Caminho do método

In [1]:
from memory_profiler import memory_usage

def medir_memoria(func, *args):
    mem_usage = memory_usage((func, args))
    return max(mem_usage) - min(mem_usage)

In [2]:
def memory_consume():
    """
    Abrindo arquivo
    """

    a = [1] * (10 ** 6)
    b = [2] * (2 * 10 ** 7)
    del b
    return a

In [3]:
mem = medir_memoria(memory_consume)
print(f"Consumo de memória do metodo: {mem} MiB")

Consumo de memória do metodo: 160.265625 MiB


Agora vamos fazer o profile do método executando ele 5 vezes

In [4]:
from cProfile import Profile
from statistics import mean
from pstats import Stats
from timeit import repeat

profile = Profile()
profile.enable()

result = repeat(memory_consume, repeat=1, number=5)
result_min = min(result)
result_max = max(result)
result_mean = mean(result)
result_total = sum(result)

result = dict(
    min=round(result_min, 4),
    max=round(result_max, 4),
    mean=round(result_mean, 4),
    total=round(result_total, 4)
)

profile.disable()
stats = Stats(profile).sort_stats("tottime")

# Armazena em um arquivo
# stats.dump_stats()

# Printa na tela
stats.print_stats()

print(result)

         212 function calls (209 primitive calls) in 0.391 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.377    0.075    0.377    0.075 /tmp/ipykernel_7514/3772902165.py:1(memory_consume)
        1    0.013    0.013    0.390    0.390 <timeit-src>:2(inner)
        9    0.000    0.000    0.000    0.000 {built-in method builtins.compile}
        1    0.000    0.000    0.000    0.000 /opt/conda/lib/python3.11/timeit.py:104(__init__)
        1    0.000    0.000    0.390    0.390 /opt/conda/lib/python3.11/timeit.py:240(repeat)
        7    0.000    0.000    0.000    0.000 /opt/conda/lib/python3.11/site-packages/traitlets/traitlets.py:708(__get__)
        7    0.000    0.000    0.000    0.000 /opt/conda/lib/python3.11/codeop.py:117(__call__)
        7    0.000    0.000    0.390    0.056 /opt/conda/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3490(run_code)
        1    0.000    0.000    0.390    0